# Output Preparation for the Nord_H2ub Spine Model

This jupyter notebook contains all routines for the preparation of the input data sources into a input data file for the model in Spine.

Authors: Johannes Giehl (jfg.eco@cbs.dk), Dana J. Hentschel (djh.eco@cbs.dk)

## General settings

### Packages:

In [1]:
import pandas as pd
import os
import openpyxl
import numpy as np

### define variables

In [2]:
excel_name = 'output_last_run'

### File paths:

In [13]:
#parent folder
parent_folder = '../02_basic_energy_model/.spinetoolbox/items/exporter/output'
folders = [f for f in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, f))]
if not folders:
    print("No folders found.")
else:
    latest_folder = max(folders, key=lambda x: os.path.getmtime(os.path.join(parent_folder, x)))
    latest_folder_path = os.path.join(parent_folder, latest_folder)
latest_folder_path = latest_folder_path.replace('\\', '/')
folder_path = latest_folder_path
folder_path += '/'

#prepared input data
output_file_path = '../02_output_data/01_basic_energy_model_outputs/'

In [14]:
#set name of the relevant files
#input file
output_old = 'Output_exported.xlsx'

## Workflow of the data preparation

### Data Import

In [21]:
df_output_raw = pd.read_excel(folder_path + output_old)
df_PV_prices = pd.read_excel('../01_input_data/02_input_prepared/Maersk_Example_Input_prepared.xlsx', sheet_name='Energy_prices')

### data frame preparation

In [34]:
#create a copy of the original output DataFrame
df_output = df_output_raw.copy()

# Combine the old header with the strings from the first three rows for each column
new_headers = df_output.columns + '_' + df_output.iloc[0] + '_' + df_output.iloc[1] + '_' + df_output.iloc[2]

# Set the new headers
df_output.columns = new_headers

# Drop the first three rows
#df = df.drop([0, 1, 2])

# Reset the index
df_output.reset_index(drop=True, inplace=True)

# Rename the first column to "timeseries"
df_output.columns.values[0] = "timeseries"

In [26]:
#calculate revenues from PV sales on the wholesale market
selected_column_name = None
for column_index in range(len(df_output.columns)):
    if df_output.iloc[0, column_index] == 'power_line_Wholesale_Kasso' \
        and df_output.iloc[1, column_index] == 'to_node' \
        and df_output.iloc[2, column_index] == 'Power_Wholesale':
        selected_column_name = df_output.columns[column_index]
        break

if selected_column_name:
    df_output['Revenue from PV'] = df_output[selected_column_name].iloc[3:] * df_PV_prices['Power_Wholesale_Out'].iloc[4]
else:
    print("Column with specified headers not found in output.")

In [27]:
total_costs = df_output['total_costs'].iloc[3]
total_proceeds = df_output['proceeds from PV'].sum()
adjusted_costs = total_costs - total_proceeds
df_output['adjusted costs'] = np.nan
df_output.at[3, 'adjusted costs'] = adjusted_costs

KeyError: 'total_costs'

### Creating one combined excel and export

In [8]:
with pd.ExcelWriter(output_file_path + excel_name + '.xlsx') as writer:
    df_output.to_excel(writer, sheet_name='Total costs')

In [28]:
df_output['proceeds from PV']

0     NaN
1     NaN
2     NaN
3    -0.0
4    -0.0
5    -0.0
6    -0.0
7    -0.0
8    -0.0
9    -0.0
10   -0.0
11   -0.0
12   -0.0
13   -0.0
14   -0.0
15   -0.0
16   -0.0
17   -0.0
18   -0.0
19   -0.0
20   -0.0
21   -0.0
22   -0.0
23   -0.0
24   -0.0
25   -0.0
Name: proceeds from PV, dtype: object

In [11]:
adjusted_costs

-94427.62279218005

In [16]:
df_output

,Unnamed: 0,connection_flow,connection_flow.1,connection_flow.2,connection_flow.3,connection_flow.4,connection_flow.5,connection_flow.6,connection_flow.7,connection_flow.8,...,unit_flow.12,unit_flow.13,unit_flow.14,unit_flow.15,unit_flow.16,unit_flow.17,unit_flow.18,unit_flow.19,unit_flow.20,unit_flow_op
0,node,pipeline_District_Heating,pipeline_District_Heating,pipeline_storage_e-methanol,pipeline_storage_e-methanol,pipeline_storage_e-methanol,pipeline_storage_e-methanol,pipeline_storage_hydrogen,pipeline_storage_hydrogen,pipeline_storage_hydrogen,...,Methanol_Reactor,Methanol_Reactor,Methanol_Reactor,Methanol_Reactor,Solar_Plant_Kasso,Steam_Plant,Steam_Plant,Steam_Plant,Steam_Plant,Electrolyzer
1,NaN,from_node,to_node,from_node,from_node,to_node,to_node,from_node,from_node,to_node,...,from_node,from_node,to_node,to_node,to_node,from_node,from_node,from_node,to_node,from_node
2,NaN,Waste_Heat,District_Heating,E-Methanol_Kasso,E-Methanol_storage_Kasso,E-Methanol_Kasso,E-Methanol_storage_Kasso,Hydrogen_Kasso,Hydrogen_storage_Kasso,Hydrogen_Kasso,...,Power_Kasso,Vaporized_Carbon_Dioxide,Raw_Methanol,Waste_Heat,Power_Kasso,Power_Kasso,Waste_Heat,Water,Steam,Power_Kasso
3,2019-01-01T00:00:00,0,0,0,0,0,0,0,0,0,...,0.1,7.167793,31.446541,7.279292,0,0,25.891118,5.178224,2.622731,NaN
4,2019-01-01T01:00:00,0,0,4.33,0,0,4.33,0,0,0,...,0.1,1.241462,5.446541,1.260773,0,0,4.484342,0.896868,0.454257,NaN
5,2019-01-01T02:00:00,0,0,0,4.33,4.33,0,11.916667,0,0,...,0.1,5.926331,26,6.018519,0,947.839122,28.177609,5.635522,2.168474,NaN
6,2019-01-01T03:00:00,0,0,28.8596,0,0,28.8596,0,11.916667,11.916667,...,0.1,11.141503,48.88,11.314815,0,947.78555,33.473906,6.694781,4.076731,NaN
7,2019-01-01T04:00:00,0,0,0,2.2352,2.2352,0,0.0,0,0,...,0.1,8.533917,37.44,8.666667,0,947.812336,30.825758,6.165152,3.122602,NaN
8,2019-01-01T05:00:00,0,0,9.7648,0,0,9.7648,0.0,0,0,...,0.1,8.533917,37.44,8.666667,0,947.812336,30.825758,6.165152,3.122602,NaN
9,2019-01-01T06:00:00,0,0,0,1.9836,1.9836,0,14.083333,0,0,...,0.1,5.452225,23.92,5.537037,0,947.843992,27.696128,5.539226,1.994996,NaN


In [29]:
df_output_raw

,Unnamed: 0,connection_flow,connection_flow.1,connection_flow.2,connection_flow.3,connection_flow.4,connection_flow.5,connection_flow.6,connection_flow.7,connection_flow.8,...,unit_flow.12,unit_flow.13,unit_flow.14,unit_flow.15,unit_flow.16,unit_flow.17,unit_flow.18,unit_flow.19,unit_flow.20,unit_flow_op
0,node,pipeline_District_Heating,pipeline_District_Heating,pipeline_storage_e-methanol,pipeline_storage_e-methanol,pipeline_storage_e-methanol,pipeline_storage_e-methanol,pipeline_storage_hydrogen,pipeline_storage_hydrogen,pipeline_storage_hydrogen,...,Methanol_Reactor,Methanol_Reactor,Methanol_Reactor,Methanol_Reactor,Solar_Plant_Kasso,Steam_Plant,Steam_Plant,Steam_Plant,Steam_Plant,Electrolyzer
1,NaN,from_node,to_node,from_node,from_node,to_node,to_node,from_node,from_node,to_node,...,from_node,from_node,to_node,to_node,to_node,from_node,from_node,from_node,to_node,from_node
2,NaN,Waste_Heat,District_Heating,E-Methanol_Kasso,E-Methanol_storage_Kasso,E-Methanol_Kasso,E-Methanol_storage_Kasso,Hydrogen_Kasso,Hydrogen_storage_Kasso,Hydrogen_Kasso,...,Power_Kasso,Vaporized_Carbon_Dioxide,Raw_Methanol,Waste_Heat,Power_Kasso,Power_Kasso,Waste_Heat,Water,Steam,Power_Kasso
3,2019-01-01T00:00:00,0,0,0,0,0,0,0,0,0,...,0.1,7.167793,31.446541,7.279292,0,0,25.891118,5.178224,2.622731,NaN
4,2019-01-01T01:00:00,0,0,4.33,0,0,4.33,0,0,0,...,0.1,1.241462,5.446541,1.260773,0,0,4.484342,0.896868,0.454257,NaN
5,2019-01-01T02:00:00,0,0,0,4.33,4.33,0,11.916667,0,0,...,0.1,5.926331,26,6.018519,0,947.839122,28.177609,5.635522,2.168474,NaN
6,2019-01-01T03:00:00,0,0,28.8596,0,0,28.8596,0,11.916667,11.916667,...,0.1,11.141503,48.88,11.314815,0,947.78555,33.473906,6.694781,4.076731,NaN
7,2019-01-01T04:00:00,0,0,0,2.2352,2.2352,0,0.0,0,0,...,0.1,8.533917,37.44,8.666667,0,947.812336,30.825758,6.165152,3.122602,NaN
8,2019-01-01T05:00:00,0,0,9.7648,0,0,9.7648,0.0,0,0,...,0.1,8.533917,37.44,8.666667,0,947.812336,30.825758,6.165152,3.122602,NaN
9,2019-01-01T06:00:00,0,0,0,1.9836,1.9836,0,14.083333,0,0,...,0.1,5.452225,23.92,5.537037,0,947.843992,27.696128,5.539226,1.994996,NaN


In [35]:
df_output

,timeseries,connection_flow_pipeline_District_Heating_from_node_Waste_Heat,connection_flow.1_pipeline_District_Heating_to_node_District_Heating,connection_flow.2_pipeline_storage_e-methanol_from_node_E-Methanol_Kasso,connection_flow.3_pipeline_storage_e-methanol_from_node_E-Methanol_storage_Kasso,connection_flow.4_pipeline_storage_e-methanol_to_node_E-Methanol_Kasso,connection_flow.5_pipeline_storage_e-methanol_to_node_E-Methanol_storage_Kasso,connection_flow.6_pipeline_storage_hydrogen_from_node_Hydrogen_Kasso,connection_flow.7_pipeline_storage_hydrogen_from_node_Hydrogen_storage_Kasso,connection_flow.8_pipeline_storage_hydrogen_to_node_Hydrogen_Kasso,...,unit_flow.12_Methanol_Reactor_from_node_Power_Kasso,unit_flow.13_Methanol_Reactor_from_node_Vaporized_Carbon_Dioxide,unit_flow.14_Methanol_Reactor_to_node_Raw_Methanol,unit_flow.15_Methanol_Reactor_to_node_Waste_Heat,unit_flow.16_Solar_Plant_Kasso_to_node_Power_Kasso,unit_flow.17_Steam_Plant_from_node_Power_Kasso,unit_flow.18_Steam_Plant_from_node_Waste_Heat,unit_flow.19_Steam_Plant_from_node_Water,unit_flow.20_Steam_Plant_to_node_Steam,unit_flow_op_Electrolyzer_from_node_Power_Kasso
0,node,pipeline_District_Heating,pipeline_District_Heating,pipeline_storage_e-methanol,pipeline_storage_e-methanol,pipeline_storage_e-methanol,pipeline_storage_e-methanol,pipeline_storage_hydrogen,pipeline_storage_hydrogen,pipeline_storage_hydrogen,...,Methanol_Reactor,Methanol_Reactor,Methanol_Reactor,Methanol_Reactor,Solar_Plant_Kasso,Steam_Plant,Steam_Plant,Steam_Plant,Steam_Plant,Electrolyzer
1,NaN,from_node,to_node,from_node,from_node,to_node,to_node,from_node,from_node,to_node,...,from_node,from_node,to_node,to_node,to_node,from_node,from_node,from_node,to_node,from_node
2,NaN,Waste_Heat,District_Heating,E-Methanol_Kasso,E-Methanol_storage_Kasso,E-Methanol_Kasso,E-Methanol_storage_Kasso,Hydrogen_Kasso,Hydrogen_storage_Kasso,Hydrogen_Kasso,...,Power_Kasso,Vaporized_Carbon_Dioxide,Raw_Methanol,Waste_Heat,Power_Kasso,Power_Kasso,Waste_Heat,Water,Steam,Power_Kasso
3,2019-01-01T00:00:00,0,0,0,0,0,0,0,0,0,...,0.1,7.167793,31.446541,7.279292,0,0,25.891118,5.178224,2.622731,NaN
4,2019-01-01T01:00:00,0,0,4.33,0,0,4.33,0,0,0,...,0.1,1.241462,5.446541,1.260773,0,0,4.484342,0.896868,0.454257,NaN
5,2019-01-01T02:00:00,0,0,0,4.33,4.33,0,11.916667,0,0,...,0.1,5.926331,26,6.018519,0,947.839122,28.177609,5.635522,2.168474,NaN
6,2019-01-01T03:00:00,0,0,28.8596,0,0,28.8596,0,11.916667,11.916667,...,0.1,11.141503,48.88,11.314815,0,947.78555,33.473906,6.694781,4.076731,NaN
7,2019-01-01T04:00:00,0,0,0,2.2352,2.2352,0,0.0,0,0,...,0.1,8.533917,37.44,8.666667,0,947.812336,30.825758,6.165152,3.122602,NaN
8,2019-01-01T05:00:00,0,0,9.7648,0,0,9.7648,0.0,0,0,...,0.1,8.533917,37.44,8.666667,0,947.812336,30.825758,6.165152,3.122602,NaN
9,2019-01-01T06:00:00,0,0,0,1.9836,1.9836,0,14.083333,0,0,...,0.1,5.452225,23.92,5.537037,0,947.843992,27.696128,5.539226,1.994996,NaN
